In [ ]:
import pandas as pd
import itertools
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error,mean_squared_error, r2_score
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
import numpy as np
run=False

In [ ]:
df_uvvis=pd.read_csv('dados.csv',index_col='Unnamed: 0').drop(['Material','group'],axis=1)
df_xrf=pd.read_csv('dados_xrf.csv',index_col='Unnamed: 0')
df_uvvis_teste=pd.read_csv('dados_teste.csv',index_col='Unnamed: 0').drop(['Material','group'],axis=1)
df_xrf_teste=pd.read_csv('dados_xrf_teste.csv',index_col='Unnamed: 0')

In [ ]:
waves=pd.read_csv('waves.csv')['Wavelength (nm)'].to_list()

In [ ]:
group_mass=['Mass']
group_ref=["ref_val_"+str(x) for x in range(3648)]
group_col=["col_val_"+str(x) for x in range(3648)]
group_col_cord=["X","Y","Z"]
group_UV_VIS=group_ref+group_col_cord#+group_col
elements=['Mg', 'Al', 'Si', 'P ', 'S ', 'Cl', 'K ', 'Ca', 'Ti', 'V ', 'Cr',
       'Mn', 'Fe', 'Co', 'Ni', 'Cu', 'Zn', 'As', 'Se', 'Rb', 'Sr', 'Y ',
       'Zr', 'Nb', 'Mo', 'Ag', 'Cd', 'Sn', 'Sb', 'Ba', 'La', 'Ce', 'Pr',
       'Nd', 'W ', 'Au', 'Hg', 'Pb', 'Bi', 'Th', 'U ', 'LE']
group_concentration_xrf=[e.strip()+' Concentration'for e in elements]
group_error_xrf=[e.strip()+' Error1s'for e in elements]
group_xrf=list(itertools.chain(*zip([e.strip()+' Concentration'for e in elements],[e.strip()+' Error1s'for e in elements])))
group_year=['Year']
group_sec=['Seculo']

In [ ]:
df=pd.concat([df_xrf,df_uvvis],axis=1)
df_teste=pd.concat([df_xrf_teste,df_uvvis_teste],axis=1)

In [ ]:
df=pd.concat([df_xrf,df_uvvis],axis=1)
df=df.drop(group_col,axis=1)
df_teste=pd.concat([df_xrf_teste,df_uvvis_teste],axis=1)
df_teste=df_teste.drop(group_col,axis=1)

import warnings
warnings.filterwarnings("ignore")

In [ ]:
df_total=pd.concat([df,df_teste],axis=0)

In [ ]:
df_total['Year'][df_total['Year']%100==0]

In [ ]:
# df_inv = (
#     pd.Series(grouped_indices).explode()                       # turn each list into its own row
#      .reset_index(name='Sample')      # bring the list-name into a column, call the values 'Sample'
#      .rename(columns={'index':'Concentration'})  # rename the old index
#      .set_index('Sample')             # make each sample ID the new index
# )
# df = df.join(df_inv)
# df.rename(columns={'Concentration':'Material_XRF'},inplace=True)
# df['Material_XRF']=df['Material_XRF'].str.split(' ').str[0]
# waves=pd.read_csv('waves.csv')['Wavelength (nm)'].to_list()

In [ ]:
import warnings
warnings.filterwarnings("ignore")

# Modelo Séculos - UVVIS + Pessagem

In [ ]:
labels='all'
df_seculos=df.copy()
df_seculos_teste=df_teste.copy()
df_seculos['Seculo']=((df['Year']//100)*100).astype(str)
df_seculos_teste['Seculo']=((df_teste['Year']//100)*100).astype(str)
if labels==3:
    df_seculos['Seculo'][(df_seculos['Seculo']=='1400')|(df_seculos['Seculo']=='1500')|(df_seculos['Seculo']=='1600')|(df_seculos['Seculo']=='1700')]='<1800'
    df_seculos['Seculo'][(df_seculos['Seculo']=='1900')|(df_seculos['Seculo']=='2000')]='>1800'
    df_seculos_teste['Seculo'][(df_seculos_teste['Seculo']=='1700')]='<1800'
    df_seculos_teste['Seculo'][(df_seculos_teste['Seculo']=='1900')|(df_seculos_teste['Seculo']=='2000')]='>1800'
df_seculos.drop(['Year'],axis=1,inplace=True)
df_seculos_teste.drop(['Year'],axis=1,inplace=True)
df_seculos = df_seculos.replace('<LOD',0)
df_seculos_teste = df_seculos_teste.replace('<LOD',0)

### Filling Missing Values

We will try different methods to fill the missings values and test each method with a decision tree using cross validation to make sure we select the method that reduces the mean squared error.

In the following cell, we see if removing the columns full of missing values improves the error of a decision tree or not. 

Since the time, or number of samples, until we get a measure is important to know the type of frequency/wave/gamma that we are working with, we cannot interpolate the missing values, since that will influence or change this important information, so we have two options: either we keep the columns with the missing values and only use machine learning models that allow us to work with missing values or we replace these missing values by 0, because that is the true value - we have missing values because we do not have a measure yet.

thres=numero_nans/numero_samples

Select all columns where the percentage of missing values is less than thres/100 %

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score, make_scorer

In [ ]:
from sklearn.svm import SVC

In [ ]:
from sklearn.metrics import f1_score
from sklearn.preprocessing import StandardScaler,MinMaxScaler,RobustScaler,MaxAbsScaler,QuantileTransformer

# Modelo Séculos - All Features

### Filling Missing Values

We will try different methods to fill the missings values and test each method with a decision tree using cross validation to make sure we select the method that reduces the mean squared error.

In the following cell, we see if removing the columns full of missing values improves the error of a decision tree or not. 

In [ ]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import StratifiedKFold, cross_validate
from sklearn.metrics import (
    make_scorer, average_precision_score, 
    roc_auc_score, f1_score, accuracy_score,
    precision_recall_curve, auc
)
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import resample

# --- assume df_seculos and df_seculos_teste already defined ---
target = 'Seculo'

# 1) Split train/test (you already have X_train, X_test, y_train, y_test)
X_train = df_seculos.drop(columns=target)
y_train = df_seculos[target]
X_test  = df_seculos_teste.drop(columns=target)
y_test  = df_seculos_teste[target]

# 2) Encode labels
lb = LabelEncoder()
y_enc_train = lb.fit_transform(y_train)
y_enc_test  = lb.transform(y_test)

# 3) Over-sample any class with fewer than 5 examples
X_over = X_train.copy()
y_over = pd.Series(y_enc_train, name='Seculo')
classes = np.unique(y_enc_train)

for cls in classes:
    idx = np.where(y_enc_train == cls)[0]
    n_missing = max(0, 5 - len(idx))
    if n_missing > 0:
        X_min = X_over.iloc[idx]
        y_min = y_over.iloc[idx]
        X_extra, y_extra = resample(
            X_min, y_min,
            replace=True, n_samples=n_missing, random_state=42
        )
        X_over = pd.concat([X_over, X_extra], axis=0)
        y_over = pd.concat([y_over, y_extra], axis=0)

# === 2) Oversampling de classes pequenas (igual ao seu código) ===
X_train_over = X_over.reset_index(drop=True).copy()
y_train_over = y_over.reset_index(drop=True).copy()

In [ ]:


# === 2) Scorers ===
scoring = {
    'au_prc': make_scorer(average_precision_score,response_method="predict_proba", average='macro'),
    'roc_auc_ovr': make_scorer(roc_auc_score,response_method="predict_proba", average='macro', multi_class='ovr'),
    'f1_macro': make_scorer(f1_score, average='macro'),
    'accuracy': make_scorer(accuracy_score)
}
# 5) Prepare results DataFrame
thresholds = [t/100 for t in range(0, 105, 5)]  # 0.0, 0.05, …, 1.0
idx = pd.Index(thresholds, name='thres')
cols = pd.MultiIndex.from_product([['Nothing','Fill0'], scoring.keys()],
                                  names=['impute','metric'])
results = pd.DataFrame(index=idx, columns=cols, dtype=float)

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# 6) Loop over thresholds and imputation
for thres in thresholds:
    # select only columns whose missing-fraction ≤ thres
    X_over_=X_over  
    mask = (X_over_.isna().sum(axis=0)/X_over_.shape[0] <= thres)
    X_sub = X_over_.loc[:, mask]
    
    for fill, label in [(False,'Nothing'), (True,'Fill0')]:
        X_in = X_sub.fillna(0) if fill else X_sub
        clf = DecisionTreeClassifier(random_state=42)
        cv_res = cross_validate(
            clf, X_in, y_over, cv=cv, scoring=scoring, return_train_score=False,error_score='raise'
        )
        
        for metric in scoring:
            results.loc[thres, (label, metric)] = cv_res[f'test_{metric}'].mean()

display(results)


In [ ]:
# import seaborn as sns
# import pandas as pd
# from sklearn.preprocessing import MinMaxScaler
# import matplotlib.pyplot as plt
# import numpy as np

# # Seleção das colunas desejadas
# columns = [('Fill0', str(e)) for e in ['au_prc', 'roc_auc_ovr', 'f1_macro', 'accuracy']]
# d = results[columns]

# # Min-Max Scaling
# scaler = MinMaxScaler()
# d_scaled = pd.DataFrame(scaler.fit_transform(d), columns=d.columns, index=d.index)

# # Criar anotações no formato "scaled (real)"
# annot = np.empty_like(d_scaled.values, dtype=object)
# for i in range(d_scaled.shape[0]):
#     for j in range(d_scaled.shape[1]):
#         scaled_val = d_scaled.iloc[i, j]
#         real_val = d.iloc[i, j]
#         annot[i, j] = f"{scaled_val:.2f} ({real_val:.3f})"

# # Plot do heatmap
# plt.figure(figsize=(10, 6))
# sns.heatmap(d_scaled, annot=annot, fmt='', cmap="viridis", cbar=True)
# plt.title("Heatmap com Valor Escalado e Real (Lado a Lado)")
# plt.show()


# 0.15

In [ ]:
mask = (X_train_over.isna().sum() / len(X_train_over)) <= 0.15
X_train_over_filled = X_train_over.loc[:, mask].fillna(0)
X_test_filled = X_test.loc[:, mask].fillna(0)

In [ ]:
X_test_filled.columns

### Scaler

In [ ]:
from sklearn.svm import SVC

In [ ]:
from sklearn.metrics import f1_score
from sklearn.preprocessing import StandardScaler,MinMaxScaler,RobustScaler,MaxAbsScaler,QuantileTransformer

In [ ]:
from sklearn.preprocessing import (
    StandardScaler, MinMaxScaler, RobustScaler,
    MaxAbsScaler, QuantileTransformer, LabelEncoder
)
from sklearn.model_selection import StratifiedKFold, cross_validate
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.metrics import (
    make_scorer,average_precision_score, roc_auc_score,
    f1_score, accuracy_score, make_scorer
)
import pandas as pd
import numpy as np

# === 1) Scalers a testar ===
scalers = {
    "No Scaler": None,
    "StandardScaler": StandardScaler(),
    "MinMaxScaler": MinMaxScaler(),
    "RobustScaler": RobustScaler(),
    "MaxAbsScaler": MaxAbsScaler(),
    "QuantileTransformer": QuantileTransformer(random_state=42)
}

# === 2) Scorers ===
scoring = {
    'au_prc': make_scorer(average_precision_score,response_method="predict_proba"),
    'roc_auc_ovr': make_scorer(roc_auc_score,response_method="predict_proba", multi_class='ovr'),
    'f1_macro': make_scorer(f1_score, average='macro'),
    'accuracy': make_scorer(accuracy_score)
}



# === 4) CV setup ===
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# === 5) Loop com Pipeline + cross_validate ===
results = pd.DataFrame(index=scalers.keys(), columns=scoring.keys(), dtype=float)

for name, scaler in scalers.items():
    # montar pipeline:
    if scaler is None:
        steps = [('clf', SVC(probability=True, random_state=42))]
    else:
        steps = [('scaler', scaler), ('clf', SVC(probability=True, random_state=42))]
    pipe = Pipeline(steps)
    
    cv_res = cross_validate(
        pipe,
        X_train_over_filled, y_train_over,
        cv=cv,
        scoring=scoring,
        return_train_score=False
    )
    # preencher resultados com médias:
    for metric in scoring:
        # cross_validate retorna chaves tipo 'test_ap_macro', etc.
        results.loc[name, metric] = np.mean(cv_res[f'test_{metric}'])

# === 6) Exibir ===
display(results)


In [ ]:
scal=QuantileTransformer(random_state=42)
X_train_over_filled_scal=scal.fit_transform(X_train_over_filled)
X_test_filled_scal=scal.transform(X_test_filled)

In [ ]:
import os
import json
import logging
import warnings
import numpy as np
import pandas as pd
from datetime import datetime
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import LabelEncoder, QuantileTransformer, OneHotEncoder
from sklearn.feature_selection import SelectKBest, f_classif, SelectFromModel, RFE
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline
from hyperopt import fmin, tpe, Trials, STATUS_OK, space_eval, hp
from sklearn.linear_model import LogisticRegression, RidgeClassifier, SGDClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
import xgboost as xgb
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix,precision_recall_curve, average_precision_score, auc
from sklearn.inspection import permutation_importance
from sklearn.metrics import make_scorer,ConfusionMatrixDisplay
import matplotlib.pyplot as plt
from sklearn.utils import resample
from sklearn.naive_bayes import GaussianNB
import random
random.seed(42)
# ----------------------
# Setup results directory
# ----------------------

RESULTS_DIR = os.path.join(os.getcwd(), 'results_models')
os.makedirs(RESULTS_DIR, exist_ok=True)

# ----------------------
# Model hyperparameter definitions
# ----------------------
class models_parameters:
#POR DENTRO DE CLASSE
    models_params = {
                    "LogisticRegression": {
                            'C': hp.choice('C', [100,250,500,1000,2500,5000]), \
                            #'class_weight': hp.choice('class_weight', ["balanced"]),\
                            'l1_ratio': hp.choice('l1_ratio', np.arange(0.1,1,0.1)),\
                            'tol': hp.choice('tol', [1*10**-50,1*10**-25,1*10**-10,1*10**-5,0.001,0.01]),\
                            'warm_start': hp.choice('warm_start', [True, False]),\
                            'fit_intercept': hp.choice('fit_intercept', [True, False]),\
                            # 'penalty':hp.choice('penalty',['elasticnet','l1', 'l2']), # - só com solver saga
                            'max_iter': hp.choice('max_iter', [100,250,500,1000]) #100
                            },
                    "GaussianProcessClassifier": {
                            'max_iter_predict': hp.choice('max_iter_predict', [5,50,100,500,1000])
                            },
                    "RandomForestClassifier": {
                            'n_estimators': hp.choice('n_estimators', [100,150,200,250,500,1000]), #110
                            'max_depth': hp.choice('max_depth', [2,3,4,5,6,7,8,9,10]),#3
                            'min_samples_split': hp.choice('min_samples_split',range(5,100,5) ), #range(20,80,2)
                            'min_samples_leaf': hp.choice('min_samples_leaf', range(10,100,10)), #30
                            #'max_features': hp.choice('max_features', range(10,100,10)),
                            'max_leaf_nodes': hp.choice('max_leaf_nodes', [2,3,4,5,6,7,8,9,10])#-
                            },
                    "GradientBoostingClassifier": {
                            # 'learning_rate': hp.choice('learning_rate', np.arange(0.1,1,0.1)),
                            # 'n_estimators': hp.choice('n_estimators', [10,50,100,200,500,1000]),\
                            # 'max_depth': hp.choice('max_depth', [2,3,4,5,6,7,8,9,10]),\
                            # 'min_samples_split': hp.choice('min_samples_split',range(10,1000,100)),
                            # 'min_samples_leaf': hp.choice('min_samples_leaf', range(10,1000,100)),\
                            # #'max_features': hp.choice('max_features', [2,4,6,8,10]),
                            # 'ccp_alpha': hp.choice('ccp_alpha', np.arange(0,10,0.1)),\
                            # 'max_leaf_nodes': hp.choice('max_leaf_nodes', [2,3,4,5,6,7,8,9,10]),
                            # 'tol': hp.choice('tol', [1*10**-50,1*10**-25,1*10**-10,1*10**-5,0.001,0.01])
                        # 'learning_rate': hp.choice('learning_rate', np.arange(0.01, 0.99, 0.01)),
                        # 'n_estimators': hp.choice('n_estimators', [100,150,200,250,300,350,400,450,500,1000]),
                        # 'max_depth': hp.choice('max_depth', [2,3,4,5,6,7,8,9,10]),
                        # 'min_samples_split': hp.choice('min_samples_split', range(5,100,5)),
                        # 'min_samples_leaf': hp.choice('min_samples_leaf',  range(10,100,10)),
                        # 'ccp_alpha': hp.choice('ccp_alpha', np.arange(0, 1, 0.01)),
                        # 'max_leaf_nodes': hp.choice('max_leaf_nodes', [None,2,3,4,5,6,7,8,9,10,20,30]),
                        # 'tol': hp.choice('tol', [1*10**-10,1*10**-5,1*10**-4, 1*10**-3, 1*10**-2,1*10**-1])
                        'learning_rate': hp.uniform('learning_rate', 0.9, 1.0),  # Use uniform distribution for learning rate
                        'n_estimators': hp.choice('n_estimators', [100, 150, 200, 250, 300,500]),
                        'max_depth': hp.choice('max_depth', [2,3, 4,]),  # Avoid overly deep trees
                        'min_samples_split': hp.choice('min_samples_split', [2, 5, 10]),  # Adjust according to dataset size
                        'min_samples_leaf': hp.choice('min_samples_leaf', [2, 5, 10]),  # Adjust according to dataset size
                        'subsample': hp.uniform('subsample', 0.8, 1.0),  # Fraction of samples used for fitting the trees
                        'max_features': hp.choice('max_features', ['sqrt', 'log2', None]),
                        'ccp_alpha': hp.choice('ccp_alpha', [0.0]),  # Complexity parameter for Minimal Cost-Complexity Pruning
                        'max_leaf_nodes': hp.choice('max_leaf_nodes', [None,2, 10, 20, 30]),
                        'tol': hp.uniform('tol', 1e-6, 1e-2),  # Tolerance for stopping criterion

                        },
                    "MLPClassifier": {
                            'hidden_layer_sizes':hp.choice('hidden_layer_sizes', [(203,124,3),(203,64,3),(203,240,120,3),(203,128,64,256,3)]),\
                            'solver':hp.choice('solver', ['adam','lbfgs','sgd']),\
                            'alpha': hp.choice('alpha', np.arange(0,1,0.1)),
                            'batch_size': hp.choice('batch_size', [10,50,100,500]),#10
                            #1*10**-25,0.00001,0.001,0.01,0.1
                            #sgd causes error
                            # 'learning_rate':hp.choice('learning_rate', ['constant','invscaling', 'adaptive']),\
                            'learning_rate_init': hp.choice('learning_rate_init', np.arange(0.1,1,0.1)),\
                            'max_iter': hp.choice('max_iter', [10,50,100,250,500,1000]),#100
                            'tol': hp.choice('tol', [1*10**-25,0.00001,0.001]),\
                            'momentum': hp.choice('momentum', [0.001,0.1,0.5,0.7])
                            },
                    "KNeighborsClassifier": {
                            # 'algorithm':hp.choice('algorithm', ['ball_tree']), \
                            'metric':hp.choice("metric", ['minkowski','cosine','euclidean','cityblock']), #
                            'n_neighbors':hp.choice('n_neighbors',[10,50,100,200]),
                            'p':hp.choice('p',[10,50,100,200]),
                            'leaf_size':hp.choice('leaf_size',[10,50,100,200]),
                            'weights':hp.choice("weights",['uniform','distance'])
                            },
                    "XGB_dart": {
                            'sample_type': hp.choice('sample_type', ["uniform", "weighted"]),
                            'normalize_type': hp.choice('normalize_type', ["tree","forest"]),\
                            'rate_drop': hp.choice('rate_drop', np.arange(0,1,0.1)),
                            'skip_drop': hp.choice('skip_drop', np.arange(0,1,0.1)),\
                            'learning_rate': hp.choice('learning_rate', np.arange(0.1,1,0.1)),
                            'max_depth': hp.choice('max_depth', [2,3,4,5,6,7,8,9,10]),
                            'min_child_weight': hp.choice('min_child_weight', np.arange(2,100,10)),
                            #'subsample': hp.choice('subsample', [1]), default is the best\
                            'gamma': hp.choice('gamma', np.arange(0,1,0.1)),
                            'colsample_bytree': hp.choice('colsample_bytree', np.arange(0.1,1,0.1)),
                            'lambda': hp.choice('lambda', np.arange(0.1,1,0.1))

                            },
                    "XGB_linear": {
                            'alpha': hp.choice('alpha', np.arange(0, 1, 0.01)), #default is better,0
                            'lambda': hp.choice('lambda', np.arange(0, 1, 0.01)),\
                            'updater': hp.choice('updater', ["shotgun","coord_descent"]),
                            'learning_rate': hp.choice('learning_rate', np.arange(0.01, 0.2, 0.01)),\
                            'feature_selector': hp.choice('feature_selector', ["cyclic", "shuffle"])
                            },
                    "XGB_tree": {
                            'learning_rate': hp.choice('learning_rate', np.arange(0.1,1,0.1)), #0.03
                            'max_depth': hp.choice('max_depth', [2,3,4,5,6,7,8,9,10]), #4
                            'gamma': hp.choice('gamma', np.arange(0,1,0.1)), #0.6
                            'lambda':hp.choice('lambda', np.arange(0,1,0.1)), #0.5
                            'min_child_weight': hp.choice('min_child_weight', np.arange(2,100,10)) #30
                            },
                    "GNB":{'var_smoothing': hp.loguniform('var_smoothing', np.log(1e-12), np.log(1e-6))},
                    "SVC": {
                        'gamma': hp.choice('epsilon', [0.0001,0.001,0.1,0.5,0.9]),#0.0001
                        'C': hp.choice('C',np.arange(10,3000,100)) #constante apartir do 2800
                        },
                    "DT": {
                            'criterion': hp.choice('criterion', ['gini', 'entropy']),
                            'max_depth': hp.choice('max_depth', np.arange(1, 20, 1)),  # Maximum depth of the tree
                            'min_samples_split': hp.choice('min_samples_split', np.arange(2, 20, 1)),  # Minimum number of samples required to split an internal node
                            'min_samples_leaf': hp.choice('min_samples_leaf', np.arange(1, 20, 1)),  # Minimum number of samples required to be at a leaf node
                            'max_features': hp.choice('max_features', ['sqrt', 'log2', None]),  # Number of features to consider when looking for the best split
                            'splitter': hp.choice('splitter', ['best', 'random']),  # Strategy used to choose the split at each node
                        }

                            }

# ----------------------
# Preprocessing
# ----------------------
def query_and_preprocess(df,df_teste):
    target = 'Seculo'
    # 1) Split train/test (you already have X_train, X_test, y_train, y_test)
    X_train = df.drop(columns=target)
    y_train = df[target]
    X_test  = df_teste.drop(columns=target)
    y_test  = df_teste[target]

    # 2) Encode labels
    lb = LabelEncoder()
    y_enc_train = lb.fit_transform(y_train)
    y_enc_test  = lb.transform(y_test)

    # 3) Over-sample any class with fewer than 5 examples
    X_over = X_train.copy()
    y_over = pd.Series(y_enc_train, name='Seculo')
    classes = np.unique(y_enc_train)

    for cls in classes:
        idx = np.where(y_enc_train == cls)[0]
        n_missing = max(0, 5 - len(idx))
        if n_missing > 0:
            X_min = X_over.iloc[idx]
            y_min = y_over.iloc[idx]
            X_extra, y_extra = resample(
                X_min, y_min,
                replace=True, n_samples=n_missing, random_state=42
            )
            X_over = pd.concat([X_over, X_extra], axis=0)
            y_over = pd.concat([y_over, y_extra], axis=0)

    # === 2) Oversampling de classes pequenas (igual ao seu código) ===
    X_train_over = X_over.reset_index(drop=True).copy()
    y_train_over = y_over.reset_index(drop=True).copy()

    X_train_droped_filled = X_train_over.loc[:, X_train_over.isna().sum(axis=0) / X_train_over.shape[0] <= 0.1].fillna(0).copy()
    X_test_droped_filled = X_test.loc[:,X_train_droped_filled.columns].fillna(0).copy()
    print(X_train_droped_filled.columns)
    
    # Scale
    scaler = QuantileTransformer(output_distribution='uniform', random_state=42)
    X_train_scaled = pd.DataFrame(scaler.fit_transform(X_train_droped_filled), columns=X_train_droped_filled.columns)
    X_test_scaled  = pd.DataFrame(scaler.transform(X_test_droped_filled),  columns=X_test_droped_filled.columns)
    return X_train_scaled, X_test_scaled, y_train_over, pd.Series(y_enc_test),lb

# Hyperopt objective with FS
# ----------------------
def hyperopt(model_name, X_train, y_train, model_space, max_evals=50):
    trials = Trials()
    n_feats = X_train.shape[1]
    fs_space = hp.choice(f'{model_name}_fs', [
        {'name': 'none'},
        {'name': 'SelectKBest',       'k': hp.quniform(f'{model_name}_k',      3, n_feats, 1)},
        {'name': 'SelectFromModel',   'threshold': hp.uniform(f'{model_name}_t', 0.01, 0.5)}
        #,{'name': 'RFE',               'n_features': hp.quniform(f'{model_name}_n', 3, n_feats, 1)}
    ])
    space = {**model_space, 'fs_method': fs_space}

    def objective(params):
        fs = params.pop('fs_method')
        # base model
        model = select_model(model_name, params)
        # apply FS
        if fs['name']=='none':
            estimator = model
        else:
            if fs['name']=='SelectKBest':
                selector = SelectKBest(f_classif,k=int(fs['k']))
            elif fs['name']=='SelectFromModel':
                selector = SelectFromModel(RandomForestClassifier(n_estimators=500, random_state=42),threshold=int(fs['threshold']))
            # else: # RFE
            #     selector = RFE(RandomForestClassifier(n_estimators=500, random_state=42),n_features_to_select=int(fs['n_features']))
            # guard
            try:
                sub = selector.fit_transform(X_train, y_train)
                if sub.ndim<2 or sub.shape[1]==0:
                    return {'loss': 0, 'status': STATUS_OK}
            except Exception as ti:
                print(ti)
                return {'loss': 0, 'status': STATUS_OK}
            estimator = Pipeline([('fs', selector),('clf', model)])
        # evaluate
        try:
            scorer=make_scorer(average_precision_score,response_method='predict_proba')
            score = cross_val_score(estimator, X_train, y_train, cv=5, scoring=scorer).mean()
            return {'loss': -score, 'status': STATUS_OK}
        except Exception as ti:
            print(ti)
            return {'loss': 0, 'status': STATUS_OK}

    best = fmin(objective, space, algo=tpe.suggest, max_evals=max_evals, trials=trials)
    return space_eval(space, best)

# ----------------------
# Model factory
# ----------------------
def select_model(name, params):
    if name=='LogisticRegression': return LogisticRegression(**params, random_state=42)
    if name=='RandomForestClassifier': return RandomForestClassifier(**params, random_state=42)
    if name=='GradientBoostingClassifier': return GradientBoostingClassifier(**params, random_state=42)
    if name=='MLPClassifier'    : return MLPClassifier(**params, random_state=42)
    if name=='KNeighborsClassifier': return KNeighborsClassifier(**params)
    if name.startswith('XGB')  :
        booster='gbtree'
        if name=='XGB_dart': booster='dart'
        if name=='XGB_linear': booster='gblinear'
        return xgb.XGBClassifier(**params, booster=booster, random_state=42)
    if name=='SVC': return SVC(**params)
    if name=='GaussianProcessClassifier': return GaussianProcessClassifier(**params, kernel=RBF(), random_state=42)
    if name=='GNB': return GaussianNB(**params)
    if name=='DT': return DecisionTreeClassifier(**params,random_state=42)
    raise ValueError(f'Unknown model: {name}')

# ----------------------
# Evaluation & Saving
# ----------------------
def save_evaluation(estimator, X_train, y_train, X_test, y_test,model_name,mname,lb):
    estimator.fit(X_train, y_train)
    preds = estimator.predict(X_test)
    probas = estimator.predict_proba(X_test)
    
    # Recover original labels

    y_test_labels = lb.inverse_transform(y_test)
    preds_labels = lb.inverse_transform(preds)
    
    # ROC AUC
    roc = average_precision_score(np.array(y_test_labels), probas)
    print(roc)
    # Classification report
    rep = classification_report(y_test_labels, preds_labels)
    txt = os.path.join(RESULTS_DIR, f'eval_{model_name}_{mname}.txt')
    with open(txt, 'w') as f:
        f.write(f'Model: {mname}\nDate: {datetime.now()}\nAU_PRC: {roc:.4f}\n')
        f.write(rep)
    
    # Permutation importance (top 20)
    r = permutation_importance(estimator, X_test, y_test, n_repeats=30, random_state=123)
    idx = np.argsort(r.importances_mean)[-20:]  # get 20 most important
    fig, ax = plt.subplots(figsize=(10, 8))
    ax.boxplot(r.importances[idx].T, vert=False, labels=X_test.columns[idx])
    ax.set_title(f'Permutation Importance - {mname}')
    perm_path = os.path.join(RESULTS_DIR, f'perm_{model_name}_{mname}.png')
    fig.tight_layout()
    fig.savefig(perm_path)
    plt.close(fig)
    
    # Confusion matrix
    cm = confusion_matrix(y_test_labels, preds_labels, labels=lb.classes_)
    fig2, ax2 = plt.subplots(figsize=(8, 6))
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=lb.classes_)
    disp.plot(ax=ax2, cmap='Blues')
    cm_path = os.path.join(RESULTS_DIR, f'cm_{model_name}_{mname}.png')
    fig2.tight_layout()
    fig2.savefig(cm_path)
    plt.close(fig2)

    return roc



# ----------------------
# Runner
# ----------------------
def run_feature_selection_pipeline(df,df_teste,model_names,start=0, max_evals=50):
    X_train,X_test,y_train,y_test,lb = query_and_preprocess(df,df_teste)
    results={}
    for name, space in list(models_parameters.models_params.items())[start:]:
        print(name)
        best = hyperopt(name, X_train, y_train, space, max_evals)
        # separate fs and params
        fs = best.pop('fs_method')
        params = best.copy()
        # build estimator
        model = select_model(name, params)
        print(fs['name'])
        if fs['name']=='none':
            estimator = model
        else:
            if fs['name']=='SelectKBest':
                selector = SelectKBest(f_classif, k=int(fs['k']))
            elif fs['name']=='SelectFromModel':
                selector = SelectFromModel(LogisticRegression(random_state=42), threshold=fs['threshold'])
            # else: # RFE
            #     selector = RFE(LogisticRegression(random_state=42), n_features_to_select=int(fs['n_features']))
            # guard
            try:
                sub = selector.fit_transform(X_train, y_train)
                if sub.ndim<2 or sub.shape[1]==0:
                    print(sub.ndim)
                    print(sub.shape)
                    score=0
            except:
                score=0
            estimator = Pipeline([('fs', selector),('clf', model)])
        # evaluate
        # try:
        scorer=make_scorer(average_precision_score,response_method='predict_proba')
        score = cross_val_score(estimator, X_train, y_train, cv=5, scoring=scorer).mean()
        # write JSON
        py_params = {k: (v.tolist() if isinstance(v, np.ndarray) else int(v) if isinstance(v, np.generic) else v)
             for k,v in params.items()}
        py_fs     = fs.tolist() if isinstance(fs, np.ndarray) else fs
        py_score  = float(score)      if isinstance(score, np.generic) else score

        out = {
            'params': py_params,
            'fs':      py_fs,
            'score':   py_score
        }

        with open(os.path.join(RESULTS_DIR, f'best_{model_name}_{name}.json'), 'w') as f:
            json.dump(out, f, indent=2)
        # final evaluation
        save_evaluation(estimator,X_train,y_train,X_test,y_test,model_name,name,lb)
        results[name]=out
        # except Exception as e:
        #     print(e)
        #     score=0
        # write JSON
        # out={'params':params,'fs':fs,'score':score}
        # try:
        #     with open(os.path.join(RESULTS_DIR,f'best_{model_name}_{name}.json'),'w') as f: json.dump(out,f,indent=2)
        # except:
        #     continue
        # # final evaluation
        # results[name]=out
    return results



In [ ]:
start=(3,0,0)
g=['uvvis','xrf','uvvis_xrf','uvvis_xrf_mass','xrf_mass']
for label in [3,4]:
    df_seculos=df.copy().drop(columns=['Material_XRF']) if 'Material_XRF' in df.columns else df.copy()
    df_seculos_teste=df_teste.copy()
    df_seculos['Seculo']=((df['Year']//100)*100).astype(str)
    df_seculos_teste['Seculo']=((df_teste['Year']//100)*100).astype(str)
    if label==3:
        print(3)
        df_seculos['Seculo'][(df_seculos['Seculo']=='1400')|(df_seculos['Seculo']=='1500')|(df_seculos['Seculo']=='1600')|(df_seculos['Seculo']=='1700')]='<1800'
        df_seculos['Seculo'][(df_seculos['Seculo']=='1900')|(df_seculos['Seculo']=='2000')]='>1800'
        df_seculos_teste['Seculo'][(df_seculos_teste['Seculo']=='1700')]='<1800'
        df_seculos_teste['Seculo'][(df_seculos_teste['Seculo']=='1900')|(df_seculos_teste['Seculo']=='2000')]='>1800'
    elif label==4:
        print(4)
        df_seculos.drop(df_seculos['Seculo'][(df_seculos['Seculo']=='1400')|(df_seculos['Seculo']=='1500')|(df_seculos['Seculo']=='1600')].index,axis=0,inplace=True)
    df_seculos.drop(['Year'],axis=1,inplace=True)
    df_seculos_teste.drop(['Year'],axis=1,inplace=True)
    df_seculos = df_seculos.replace('<LOD',0)
    df_seculos_teste = df_seculos_teste.replace('<LOD',0)
    #a=df_seculos.copy().groupby('Seculo').apply(lambda x: x.sample(n=5, random_state=42)).reset_index(drop=True)

    for e,ename in list(zip([group_UV_VIS+group_sec,group_xrf+group_sec,group_UV_VIS+group_xrf+group_sec,group_UV_VIS+group_xrf+group_mass+group_sec,group_xrf+group_mass+group_sec],['uvvis','xrf','uvvis_xrf','uvvis_xrf_mass','xrf_mass'])):
    #for e,ename in list(zip([group_xrf+group_sec,group_xrf+group_mass+group_sec],['xrf','xrf_mass'])):
        model_name=f'{str(label)}labels_{ename}'
        if label==start[0] and ename == g[start[1]]:
            a=run_feature_selection_pipeline(df_seculos[e],df_seculos_teste[e],model_name,start[2], max_evals=50) #final train and then save_evaluation
        elif label==start[0] and ename in g[start[1]+1:]:
            run_feature_selection_pipeline(df_seculos[e],df_seculos_teste[e],model_name, max_evals=50)
        elif label>start[0]:
            run_feature_selection_pipeline(df_seculos[e],df_seculos_teste[e],model_name, max_evals=50)